In [13]:
import datetime
import fetcher
import pandas as pd
from finta import TA
import numpy as np

startDate = datetime.date(2023, 12, 1)
endDate = datetime.date(2024, 6, 19)
#endDate = datetime.date.today()
Stocks_list = ["NOW"]

# Retrieving tickers from a file
stocks = pd.read_csv("SCANS/US High ADR_2024-06-28.csv")
#Neglecting tickers with Exchange values "AMEX" and "OTC"
stocks = stocks[~stocks["Exchange"].isin(["OTC", "AMEX"])]

#for ticker in stocks["Symbol"].tolist()[1:]:
#    Stocks_list.append(ticker)


focus_list_vol = {}
focus_list_ADR = {}
focus_list_3Month = {}
focus_list_1Month = {}
focus_list_RS_min_max = {}
focus_list_EMA10 = []
focus_candleSize = {}
focus_SDT = []
focus_GDR = []
focus_TB = []

index_df = fetcher.__download_data( ["SPY"], startDate, endDate, "US")

for ticker in Stocks_list:

    #Neglecting tickers with .A or .B or "/" in their names
    if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
        continue
    df = fetcher.__download_data([ticker], startDate, endDate, "US")
    #Neglecting tickers with len < 66
    if len(df) < 30:
        continue
    
    # Avg Volume*Price
    df_temp = df.iloc[-30:]
    avg_dollar_vol = 0
    for index, row in df_temp.iterrows():
        avg_dollar_vol += (
            df_temp.loc[index, "Volume"] * df_temp.loc[index, "Close"]
        ) / 50
    if avg_dollar_vol >= 20000000:
        focus_list_vol[ticker] = avg_dollar_vol
 
    # ADR%
    df_temp = df.iloc[-20:]
    sum = 0
    for index, row in df_temp.iterrows():
        sum += (df_temp.loc[index, "High"] / df_temp.loc[index, "Low"]) / 20

    ADR = 100 * (sum - 1)
    if ADR >= 2.25:
        focus_list_ADR[ticker] = ADR
   
    # Low near to or less than EMA10
    if df["Low"][-1] <= (1+ADR/300)*(TA.EMA(df, period = 10)[-1]):
        focus_list_EMA10.append(ticker)

    #Check candle size less than ADR%
    stopLoss = -1*(df["Low"][-1]/df["High"][-1] -1)*100
    if stopLoss <= ADR*1.5 and stopLoss <= 7: #
        focus_candleSize[ticker] = stopLoss
      
    """# 1Month% performance
    df_temp = df.iloc[-22:]
    df_low = df_temp["Low"]
    df_high = df_temp["High"]
    low = df_low[0]
    high = df_high[1]
    max_change_perc = 0
    for i in range(len(df_low)-1):
        for j in range(i, len(df_high)):
            change_perc = (df_high[j]-df_low[i])/df_low[i] * 100
            if change_perc > max_change_perc:
                max_change_perc = change_perc
        
    focus_list_1Month[ticker] = max_change_perc


    # 3Month% performance
    
    df_temp = df.iloc[-66:]
    df_low = df_temp["Low"]
    df_high = df_temp["High"]
    low = df_low[0]
    high = df_high[1]
    max_change_perc = 0
    for i in range(len(df_low)-1):
        for j in range(i, len(df_high)):
            change_perc = (df_high[j]-df_low[i])/df_low[i] * 100
            if change_perc > max_change_perc:
                max_change_perc = change_perc
        
    focus_list_3Month[ticker] = max_change_perc"""

    #SDT
    df_temp = df.iloc[-66:]

    low, high, close, Open = df_temp.iloc[-1]["Low"], df_temp.iloc[-1]["High"], df_temp.iloc[-1]["Close"], df_temp.iloc[-1]["Open"]
    Range = high - low

    if low < df_temp.iloc[-2]["Low"] and Open >= (low + .25*Range):
        if close < Open:
            if  Open - close <= .3*(Range)  and close >=  (low + .2*Range):
                focus_SDT.append(ticker)

        else:
            focus_SDT.append(ticker)


    #GDR
    if Open < df_temp.iloc[-2]["Close"] and Open <= ( low + .25*Range) and close > (low + .5*Range):# Open:#(low + .75*Range):
        focus_GDR.append(ticker)

    #Reversal

    #TB
    
    if low > df_temp.iloc[-2]["Low"] and high < df_temp.iloc[-2]["High"]:
        if close < Open:
            if Open - close <= .5*Range and close >=  (low + .2*Range):
                focus_TB.append(ticker)
        else:
            focus_TB.append(ticker)

    else:
        if Range <= 0.85*(df_temp.iloc[-2]["High"]-df_temp.iloc[-2]["Low"]):
            if close < Open:
                if Open - close <= .5*Range and close >=  (low + .2*Range):
                    focus_TB.append(ticker)
            else:
                focus_TB.append(ticker)


    # Relative Strength low-high ratio 1M
    """index_df_temp = index_df.iloc[-22:]
    df_temp = df.iloc[-22:]
    RS = df_temp["Close"].values/index_df_temp["Close"].values
    min = RS[0]
    max = RS[1]
    for i in range(len(RS)-1):
        for j in range(i+1, len(RS)):
            if RS[j] > RS[i]:
                if (RS[j]-RS[i])/RS[i] * 100 > (max-min)/min *100:
                    max = RS[j]
                    min = RS[i]

    RS_change_perc = (max-min)/min * 100

    focus_list_RS_min_max[ticker] = RS_change_perc"""
    
final_focus = set(focus_list_ADR.keys()) & set(focus_list_vol.keys())  & set(focus_candleSize) #& set(focus_list_EMA10)
print(final_focus)

fetching data from API for SPY
[*********************100%***********************]  1 of 1 completed
fetching data from API for NOW
[*********************100%***********************]  1 of 1 completed
{'NOW'}


In [14]:
focus = set(focus_list_ADR.keys()) & set(focus_list_vol.keys()) & set(focus_SDT + focus_GDR + focus_TB)#set(focus_candleSize.keys()) # & set(focus_list_EMA10)
#focus = set(focus_list_ADR.keys()) & set(focus_list_vol.keys()) & set(focus_GDR)
ADR_dict = {}
avg_dollar_vol_dict = {}
#Perf_3M = {}
#RS_min_max = {}
#Perf_1M = {}


for ticker in focus:
    ADR_dict[ticker] = focus_list_ADR[ticker]
    avg_dollar_vol_dict[ticker] = focus_list_vol[ticker]
    #Perf_3M[ticker] = focus_list_3Month[ticker]
    #RS_min_max[ticker] = focus_list_RS_min_max[ticker]
    #Perf_1M[ticker] = focus_list_1Month[ticker]
    

In [15]:
len(focus)

1

## Grouping by Industries

In [5]:
industries = {}
for ticker in focus:
    if stocks.loc[stocks['Symbol'] == ticker]["Industry"].values[0] not in industries.keys():
        industries[stocks.loc[stocks['Symbol'] == ticker]["Industry"].values[0]] = []
    industries[stocks.loc[stocks['Symbol'] == ticker]["Industry"].values[0]].append(ticker)

industries = dict(sorted(industries.items(), key=lambda i: -len(i[1])))

## Ranking by dol_vol for each inds

In [6]:
for inds in industries:
    avg_dollar_vol_dict_tmp = {}
    for ticker in industries[inds]:
        avg_dollar_vol_dict_tmp[ticker] = avg_dollar_vol_dict[ticker]
    industries[inds] = [ticker for ticker, _ in sorted(avg_dollar_vol_dict_tmp.items(), key=lambda item: item[1], reverse=True)]

In [7]:
bocus = []
for inds in industries:
    bocus.extend(industries[inds])
print(bocus)

['ADBE', 'NOW', 'PLTR', 'ADSK', 'DDOG', 'HUBS', 'APP', 'SNAP', 'RBLX', 'OKTA', 'TOST', 'MNDY', 'AI', 'NTNX', 'IOT', 'DOCU', 'CYBR', 'CFLT', 'ALTR', 'CVLT', 'ZETA', 'NCNO', 'KRYS', 'ELAN', 'TECH', 'MRUS', 'HALO', 'AXSM', 'DYN', 'JANX', 'ONON', 'RL', 'CROX', 'SKX', 'LEVI', 'OXM', 'KTB', 'CLSK', 'WULF', 'HUT', 'BITF', 'CIFR', 'SRPT', 'INSM', 'IONS', 'PTCT', 'NET', 'WIX', 'CRDO', 'VRNS', 'PHM', 'TOL', 'KBH', 'WSM', 'CASY', 'ABG', 'ETN', 'AYI', 'POWL', 'ICLR', 'FOUR', 'ABM', 'DKNG', 'CNK', 'SIX', 'ICUI', 'PRCT', 'LIVN', 'BURL', 'TPR', 'BOOT', 'FTAI', 'SPR', 'AVAV', 'ASML', 'SWKS', 'ASX', 'MAS', 'SPXC', 'VAL', 'SDRL', 'CLS', 'CAMT', 'SPOT', 'TME', 'EXR', 'VNO', 'ZIM', 'MATX', 'OLLI', 'M', 'DELL', 'HPE', 'EDU', 'PLNT', 'IBP', 'FLR', 'COIN', 'TW', 'KGC', 'FSM', 'SAIA', 'XPO', 'CCL', 'NCLH', 'VRT', 'FLEX', 'CARR', 'ACLS', 'PHIN', 'ATMU', 'CMG', 'SG', 'GFL', 'IP', 'CHWY', 'CPB', 'ANET', 'ASTS', 'LPX', 'HIMS', 'SPB', 'GME', 'BAP', 'ELF', 'UAL', 'CRH', 'PR', 'VST']


In [8]:
industries

{'Packaged Software': ['ADBE',
  'NOW',
  'PLTR',
  'ADSK',
  'DDOG',
  'HUBS',
  'APP',
  'SNAP',
  'RBLX',
  'OKTA',
  'TOST',
  'MNDY',
  'AI',
  'NTNX',
  'IOT',
  'DOCU',
  'CYBR',
  'CFLT',
  'ALTR',
  'CVLT',
  'ZETA',
  'NCNO'],
 'Biotechnology': ['KRYS',
  'ELAN',
  'TECH',
  'MRUS',
  'HALO',
  'AXSM',
  'DYN',
  'JANX'],
 'Apparel/Footwear': ['ONON', 'RL', 'CROX', 'SKX', 'LEVI', 'OXM', 'KTB'],
 'Data Processing Services': ['CLSK', 'WULF', 'HUT', 'BITF', 'CIFR'],
 'Pharmaceuticals: Major': ['SRPT', 'INSM', 'IONS', 'PTCT'],
 'Information Technology Services': ['NET', 'WIX', 'CRDO', 'VRNS'],
 'Homebuilding': ['PHM', 'TOL', 'KBH'],
 'Specialty Stores': ['WSM', 'CASY', 'ABG'],
 'Electrical Products': ['ETN', 'AYI', 'POWL'],
 'Miscellaneous Commercial Services': ['ICLR', 'FOUR', 'ABM'],
 'Movies/Entertainment': ['DKNG', 'CNK', 'SIX'],
 'Medical Specialties': ['ICUI', 'PRCT', 'LIVN'],
 'Apparel/Footwear Retail': ['BURL', 'TPR', 'BOOT'],
 'Aerospace & Defense': ['FTAI', 'SPR', 'AVAV

In [ ]:
["Wholesale Distributors", "Department Stores", "Internet Retail", "Specialty Stores", "Home Improvement Chains", ""]

In [ ]:
["Information Technology Services", "Internet Software/Services"]

In [ ]:
["Restaurants", "Broadcasting", "Casinos/Gaming","Cable/Satellite TV", "Hotels/Resorts/Cruise lines", "Movies/Entertainment", "Other Consumer Services", "Restaurants", ""]

In [ ]:
["Computer Processing Hardware", "Semiconductors"]

In [ ]:
["Electronic Components", "Telecommunications Equipment", "Electronic Production Equipment", "Electronic Equipment/Instruments", "Computer Peripherals"]

In [ ]:
["Industrial Machinery", "Trucks/Construction/Farm Machinery"]

In [ ]:
["Engineering & Construction", "Homebuilding", "Building Products", "Construction Materials"]

In [77]:
sectors = {}
for ticker in Stocks_list:
    sector = stocks.loc[stocks['Symbol'] == ticker]["Sector"].values[0]
    industry = stocks.loc[stocks['Symbol'] == ticker]["Industry"].values[0]
    if sector not in sectors.keys():
        sectors[sector] = []
    if industry not in sectors[sector]:
        sectors[sector].append(industry)

In [78]:
sectors

{'Retail Trade': ['Apparel/Footwear Retail',
  'Specialty Stores',
  'Department Stores',
  'Internet Retail',
  'Electronics/Appliance Stores',
  'Food Retail',
  'Discount Stores',
  'Home Improvement Chains',
  'Drugstore Chains'],
 'Technology Services': ['Internet Software/Services',
  'Data Processing Services',
  'Packaged Software',
  'Information Technology Services'],
 'Electronic Technology': ['Aerospace & Defense',
  'Electronic Components',
  'Semiconductors',
  'Telecommunications Equipment',
  'Computer Peripherals',
  'Computer Processing Hardware',
  'Electronic Production Equipment',
  'Electronic Equipment/Instruments'],
 'Health Technology': ['Biotechnology',
  'Pharmaceuticals: Major',
  'Medical Specialties',
  'Pharmaceuticals: Other',
  'Pharmaceuticals: Generic'],
 'Finance': ['Finance/Rental/Leasing',
  'Real Estate Development',
  'Investment Managers',
  'Investment Banks/Brokers',
  'Major Banks',
  'Real Estate Investment Trusts',
  'Regional Banks',
  'Mu

In [ ]:
["Retail Trade", ]

In [ ]:
["Packaged Software", "Aerospace & Defense", "Semiconductors"]